<a href="https://colab.research.google.com/github/olexandr-choorsin/ab-test-analysis/blob/main/Automated_A_B_Testing_Framework_Statistical_Significance_%26_Dimensional_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Interactive Tableau Dashboard

You can find the interactive visualization of these A/B test results here:
[A/B Test Performance Matrix](https://public.tableau.com/views/ABTestMasteryStatisticalSignificanceAnalysis/Dashboard1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)

Data Loading and Initial Environment Setup
This section establishes the connection to Google BigQuery, authenticates the user, and extracts the raw dataset using a comprehensive SQL query. The goal is to collect session-level data, including user dimensions (country, device) and key conversion events (orders, checkouts, account creations).

In [ ]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from google.colab import auth

# --- Environment Authentication ---
# Authenticating the user to access BigQuery datasets
auth.authenticate_user()

# Configuring the BigQuery client for the specific project
project_id = 'data-analytics-mate'
client = bigquery.Client(project=project_id)

# --- SQL Data Extraction ---
# This query consolidates session info with events, orders, and account creations.
# It uses CTEs (Common Table Expressions) to organize data by user dimensions.

sql_query = """
WITH session_info AS
(

SELECT
  s.date,
  s.ga_session_id,
  sp.country,
  sp.device,
  sp.continent,
  sp.channel,
  ab.test,
  ab.test_group
FROM `data-analytics-mate.DA.ab_test` ab
JOIN `data-analytics-mate.DA.session` s ON s.ga_session_id = ab.ga_session_id
JOIN `data-analytics-mate.DA.session_params` sp ON sp.ga_session_id = ab.ga_session_id
),

session_with_orders AS
(
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  COUNT(DISTINCT o.ga_session_id) AS session_with_orders
FROM `data-analytics-mate.DA.order` o
JOIN session_info ON o.ga_session_id = session_info.ga_session_id
GROUP BY
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
),

events AS
(
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  ep.event_name,
  COUNT(DISTINCT ep.ga_session_id) AS event_cnt
FROM `data-analytics-mate.DA.event_params` ep
JOIN session_info ON ep.ga_session_id = session_info.ga_session_id
GROUP BY
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  ep.event_name
),

session AS
(
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
FROM session_info
GROUP BY
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
),

account AS
(
SELECT
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group,
  COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
FROM `data-analytics-mate.DA.account_session` acs
JOIN session_info ON acs.ga_session_id = session_info.ga_session_id
GROUP BY
  session_info.date,
  session_info.country,
  session_info.device,
  session_info.continent,
  session_info.channel,
  session_info.test,
  session_info.test_group
)

SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  "session with orders" AS event_name,
  session_with_orders.session_with_orders AS value
FROM session_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  event_name,
  event_cnt AS value
FROM events
UNION ALL
SELECT
  session.date,
  session.country,
  session.device,
  session.continent,
  session.channel,
  session.test,
  session.test_group,
  "session" AS event_name,
  session_cnt AS value
FROM session
UNION ALL
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  "new account" AS event_name,
  new_account_cnt AS value
FROM account
"""

# --- Data Loading and Type Casting ---
# Loading the query results into a Pandas DataFrame
df = client.query(sql_query).to_dataframe()

# Ensure the 'date' column is in the correct datetime format
df['date'] = pd.to_datetime(df['date'])

# Exporting raw data for reproducibility and GitHub documentation
df.to_csv('online_store_raw_data.csv', index=False)

# Quick verification of data types and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        800996 non-null  datetime64[ns]
 1   country     800996 non-null  object        
 2   device      800996 non-null  object        
 3   continent   800996 non-null  object        
 4   channel     800996 non-null  object        
 5   test        800996 non-null  Int64         
 6   test_group  800996 non-null  Int64         
 7   event_name  800996 non-null  object        
 8   value       800996 non-null  Int64         
dtypes: Int64(3), datetime64[ns](1), object(5)
memory usage: 57.3+ MB


Data Transformation: Pivoting for Metric Analysis
In this section, we transform the raw long-format data into a Wide Format. This structure allows us to treat each user interaction (e.g., view_item, add_to_cart) as a separate column, which is essential for calculating conversion rates and performing segment-based A/B testing.

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

# --- Step 1: Pivoting the Data ---
# Aggregating values to align events as columns for each unique segment.
# We group by test parameters and dimensions to isolate specific user cohorts.
df_pivot = df.pivot_table(
    index=['test', 'test_group', 'country', 'device', 'continent', 'channel'],
    columns='event_name',
    values='value',
    aggfunc='sum'
).reset_index()

# --- Step 2: Data Cleaning (Handling Missing Values) ---
# Replacing NaNs with 0 for segments where specific events did not occur.
# This ensures mathematical consistency for future calculations.
df_pivot = df_pivot.fillna(0)

# --- Step 3: Verification ---
# Inspecting the new column structure and the first few rows of the pivoted dataset.
print("Pivoted Columns:", df_pivot.columns.tolist())
df_pivot.head()

Pivoted Columns: ['test', 'test_group', 'country', 'device', 'continent', 'channel', 'add_payment_info', 'add_shipping_info', 'add_to_cart', 'begin_checkout', 'click', 'first_visit', 'new account', 'page_view', 'scroll', 'select_item', 'select_promotion', 'session', 'session with orders', 'session_start', 'user_engagement', 'view_item', 'view_item_list', 'view_promotion', 'view_search_results']


event_name,test,test_group,country,device,continent,channel,add_payment_info,add_shipping_info,add_to_cart,begin_checkout,...,select_item,select_promotion,session,session with orders,session_start,user_engagement,view_item,view_item_list,view_promotion,view_search_results
0,1,1,(not set),desktop,(not set),Direct,0,0,0,0,...,0,0,15,2,15,12,4,0,6,1
1,1,1,(not set),desktop,(not set),Organic Search,0,0,0,0,...,0,0,19,3,19,16,3,0,7,1
2,1,1,(not set),desktop,(not set),Paid Search,0,1,0,1,...,0,1,8,2,8,7,3,0,3,1
3,1,1,(not set),desktop,(not set),Social Search,0,0,0,0,...,0,0,9,3,9,6,3,0,4,0
4,1,1,(not set),desktop,(not set),Undefined,0,0,0,0,...,0,0,4,1,4,4,1,0,1,0


Step 1: Universal Long Format Preparation (Multi-Dimension Aggregation)

In this stage, we iterate through various business dimensions (Country, Device, Channel, Continent) to create a unified dataset. By standardizing the dimension names into dimension_name and dimension_value, we enable a scalable analysis where all segments are processed using the same statistical logic.

In [ ]:
# Define the list of dimensions for granular analysis
all_dimensions = ['country', 'device', 'channel', 'continent']
frames = []

for dim in all_dimensions:
    # --- 1. Dimensional Aggregation ---
    # Summing up sessions and key conversion events for each specific dimension
    temp = df_pivot.groupby(['test', 'test_group', dim]).sum(numeric_only=True).reset_index()

    # --- 2. Schema Standardization ---
    # Renaming the specific dimension column (e.g., 'country') to a generic 'dimension_value'
    # This allows us to concatenate different dimensions into a single table
    temp = temp.rename(columns={dim: 'dimension_value'})

    # --- 3. Metadata Tagging ---
    # Adding a label to identify the source dimension (e.g., tagging 'USA' as 'country')
    temp['dimension_name'] = dim

    # Selecting core metrics and identifiers for the final analysis
    # Ensure event names match exactly with your pivoted columns
    target_columns = [
        'test', 'test_group', 'dimension_name', 'dimension_value', 'session',
        'add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account'
    ]

    temp = temp[target_columns]
    frames.append(temp)

# --- 4. Final Consolidation ---
# Combining all individual dimension frames into one master dataset (df_all_dims)
df_all_dims = pd.concat(frames, ignore_index=True)

# Visualizing the result of the aggregation
display(df_all_dims.head())

event_name,test,test_group,dimension_name,dimension_value,session,add_payment_info,add_shipping_info,begin_checkout,new account
0,1,1,country,(not set),369,6,16,16,29
1,1,1,country,Albania,9,1,2,2,0
2,1,1,country,Algeria,29,1,1,1,1
3,1,1,country,Argentina,122,2,3,3,6
4,1,1,country,Armenia,8,0,1,1,1


Step 2: Statistical "Decoder" (Massive Z-Testing)This section executes the core statistical logic.

We iterate through every unique segment (Dimension + Metric combination) to determine if the difference between the Control and Test groups is statistically significant. Using the Z-test for proportions, we calculate the $p\text{-value}$ and apply a standard significance threshold of $\alpha = 0.05$.

In [ ]:
results = []
metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "new account"]

# Identifying all unique segment combinations (Test + Dimension + Value)
pairs = df_all_dims[['test', 'dimension_name', 'dimension_value']].drop_duplicates()

# --- Statistical Processing Loop ---
for _, row in pairs.iterrows():
    # 1. Filtering data for the specific segment (Control vs Test)
    temp_data = df_all_dims[
        (df_all_dims['test'] == row['test']) &
        (df_all_dims['dimension_name'] == row['dimension_name']) &
        (df_all_dims['dimension_value'] == row['dimension_value'])
    ]

    # 2. Integrity Check: Ensure both groups exist in the segment
    if len(temp_data) < 2:
        continue

    for m in metrics:
        # 3. Data Extraction: Conversions (count) and Sessions (nobs) for each group
        # Group 1 = Control, Group 2 = Test
        c1 = temp_data[temp_data['test_group'] == 1][m].values[0]
        n1 = temp_data[temp_data['test_group'] == 1]['session'].values[0]
        c2 = temp_data[temp_data['test_group'] == 2][m].values[0]
        n2 = temp_data[temp_data['test_group'] == 2]['session'].values[0]

        # 4. Error Handling & Statistical Calculation
        # Assign p-value = 1 if data is insufficient for a valid test
        if n1 == 0 or n2 == 0 or (c1 == 0 and c2 == 0):
            p_val = 1
        else:
            # Running the Z-test for proportions
            # Returns: z_stat (ignored), p_value
            _, p_val = proportions_ztest([c1, c2], [n1, n2])

        # 5. Storing Analytical Findings
        results.append({
            'test': row['test'],
            'dimension_name': row['dimension_name'],
            'dimension_value': row['dimension_value'],
            'metric': m,
            'p_value': p_val,
            'significant': p_val < 0.05  # Standard significance threshold (95% Confidence)
        })

# --- Final Results Consolidation ---
df_final_results = pd.DataFrame(results)

display(df_final_results.head())

,test,dimension_name,dimension_value,metric,p_value,significant
0,1,country,(not set),add_payment_info,0.984979,False
1,1,country,(not set),add_shipping_info,0.687014,False
2,1,country,(not set),begin_checkout,0.687014,False
3,1,country,(not set),new account,0.856983,False
4,1,country,Albania,add_payment_info,0.173568,False


Step 3: The Grand Merge (Data Alignment)

To perform side-by-side comparison and calculate business metrics (like Uplift), we need to restructure our data. In this section, we split the consolidated dataset into Control and Test groups and then perform an inner join. This creates a "Wide Format" where each row represents a unique segment with all its corresponding metrics from both test groups.

In [ ]:
# --- 1. Splitting the Groups ---
# Segregating data into distinct dataframes based on the test_group ID.
# Group 1: Control (Baseline)
# Group 2: Test (Variation)
control = df_all_dims[df_all_dims['test_group'] == 1].copy()
test = df_all_dims[df_all_dims['test_group'] == 2].copy()

# --- 2. Side-by-Side Consolidation ---
# Merging the groups on shared dimensions (test, dimension name, and value).
# We use suffixes ('_cntrl' and '_test') to clearly distinguish the columns.
df_wide = pd.merge(
    control,
    test,
    on=['test', 'dimension_name', 'dimension_value'],
    suffixes=('_cntrl', '_test')
)

# Display the resulting wide structure
df_wide.head()

event_name,test,test_group_cntrl,dimension_name,dimension_value,session_cntrl,add_payment_info_cntrl,add_shipping_info_cntrl,begin_checkout_cntrl,new account_cntrl,test_group_test,session_test,add_payment_info_test,add_shipping_info_test,begin_checkout_test,new account_test
0,1,1,country,(not set),369,6,16,16,29,2,373,6,14,14,28
1,1,1,country,Albania,9,1,2,2,0,2,16,0,0,0,1
2,1,1,country,Algeria,29,1,1,1,1,2,23,1,1,1,0
3,1,1,country,Argentina,122,2,3,3,6,2,122,2,5,5,5
4,1,1,country,Armenia,8,0,1,1,1,2,11,0,0,0,1


Step 4: Reshaping Metrics (Control Group Normalization)

In this stage, we begin the process of unpivoting the data using the melt function. The goal is to isolate the Control group metrics and clean their identifiers. By stripping technical suffixes like _cntrl, we create a standardized metric column that will allow us to perfectly align Control data with Test data in the final assembly.

In [ ]:
# --- 1. Melting the Control Metrics ---
# Transforming wide metric columns into a long-list format.
# This aligns multiple conversion events into a single 'metric' column.
df_melted_cntrl = df_wide.melt(
    id_vars=['test', 'dimension_name', 'dimension_value', 'session_cntrl'],
    value_vars=[
        'add_payment_info_cntrl', 'add_shipping_info_cntrl',
        'begin_checkout_cntrl', 'new account_cntrl'
    ],
    var_name='metric_raw',
    value_name='conversions_cntrl'
)

# --- 2. String Cleaning ---
# Removing the technical suffix '_cntrl' to create a clean 'metric' name.
# This ensures that 'add_payment_info_cntrl' simply becomes 'add_payment_info'.
df_melted_cntrl['metric'] = df_melted_cntrl['metric_raw'].str.replace('_cntrl', '', regex=False)

# Display a sample of the normalized control data
df_melted_cntrl.head()

,test,dimension_name,dimension_value,session_cntrl,metric_raw,conversions_cntrl,metric
0,1,country,(not set),369,add_payment_info_cntrl,6,add_payment_info
1,1,country,Albania,9,add_payment_info_cntrl,1,add_payment_info
2,1,country,Algeria,29,add_payment_info_cntrl,1,add_payment_info
3,1,country,Argentina,122,add_payment_info_cntrl,2,add_payment_info
4,1,country,Armenia,8,add_payment_info_cntrl,0,add_payment_info


Step 5: Reshaping Metrics (Test Group Normalization)

Following the same logic as the Control group, we now unpivot the Test group metrics. By isolating session_test and conversions_test and cleaning the metric labels, we prepare the second half of our dataset for the final comparative analysis.

In [ ]:
# --- 1. Melting the Test Metrics ---
# Transforming wide test metric columns into long-format rows.
# Each conversion event is now labeled as a generic 'metric_raw'.
df_melted_test = df_wide.melt(
    id_vars=['test', 'dimension_name', 'dimension_value', 'session_test'],
    value_vars=[
        'add_payment_info_test', 'add_shipping_info_test',
        'begin_checkout_test', 'new account_test'
    ],
    var_name='metric_raw',
    value_name='conversions_test'
)

# --- 2. Label Standardizing ---
# Stripping the '_test' suffix to align metric names with the Control group.
# This ensures consistency: 'begin_checkout_test' becomes 'begin_checkout'.
df_melted_test['metric'] = df_melted_test['metric_raw'].str.replace('_test', '', regex=False)

# Quick look at the transformed test data
df_melted_test.head()

,test,dimension_name,dimension_value,session_test,metric_raw,conversions_test,metric
0,1,country,(not set),373,add_payment_info_test,6,add_payment_info
1,1,country,Albania,16,add_payment_info_test,0,add_payment_info
2,1,country,Algeria,23,add_payment_info_test,1,add_payment_info
3,1,country,Argentina,122,add_payment_info_test,2,add_payment_info
4,1,country,Armenia,11,add_payment_info_test,0,add_payment_info


Step 6: Final Integration (Merging Metrics & Statistical Results)

In this final assembly stage, we perform a multi-key join to combine our reshaped metrics with the statistical $p\text{-values}$ calculated earlier. By merging on the composite key of test, dimension, and metric, we create a comprehensive "Long Format" dataset. This table serves as the single source of truth, containing both raw volumes and statistical significance markers for every segment.

In [ ]:
# --- 1. Consolidating Group Metrics ---
# Merging cleaned Control and Test dataframes.
# We drop 'metric_raw' to keep only the standardized 'metric' labels.
df_stats = pd.merge(
    df_melted_cntrl.drop(columns='metric_raw'),
    df_melted_test.drop(columns='metric_raw'),
    on=['test', 'dimension_name', 'dimension_value', 'metric']
)

# --- 2. Integrating Statistical Significance ---
# Attaching the Z-test results (p_value and significance flag) to our metric table.
# This alignment ensures each conversion point has a corresponding math validation.
df_final_long = pd.merge(
    df_stats,
    df_final_results,
    on=['test', 'dimension_name', 'dimension_value', 'metric']
)

# Preview the unified analytical dataset
display(df_final_long.head())

,test,dimension_name,dimension_value,session_cntrl,conversions_cntrl,metric,session_test,conversions_test,p_value,significant
0,1,country,(not set),369,6,add_payment_info,373,6,0.984979,False
1,1,country,Albania,9,1,add_payment_info,16,0,0.173568,False
2,1,country,Algeria,29,1,add_payment_info,23,1,0.866954,False
3,1,country,Argentina,122,2,add_payment_info,122,2,1.000000,False
4,1,country,Armenia,8,0,add_payment_info,11,0,1.000000,False


Step 7: Business Metrics & Final Export

In this concluding stage, we calculate the core KPIs that drive business decisions: Conversion Rate (CR) for both groups and the Relative Uplift. These metrics translate statistical findings into business value. Finally, we export the refined dataset into a CSV file, ready for visualization in Tableau or Power BI.

In [ ]:
# --- 1. Final Schema Definition ---
# Selecting a logical column order to ensure the report is easy to read.
cols_to_keep = [
    'test', 'dimension_name', 'dimension_value', 'metric',
    'session_cntrl', 'conversions_cntrl',
    'session_test', 'conversions_test',
    'p_value', 'significant'
]
df_final = df_final_long[cols_to_keep].copy()

# --- 2. Calculating Business KPIs ---
# Calculating Conversion Rates (CR) for Control and Test groups
df_final['cr_cntrl'] = df_final['conversions_cntrl'] / df_final['session_cntrl']
df_final['cr_test'] = df_final['conversions_test'] / df_final['session_test']

# Calculating Relative Uplift (Percentage change from Control to Test)
# Formula: (CR_test - CR_control) / CR_control
df_final['uplift'] = (df_final['cr_test'] - df_final['cr_cntrl']) / df_final['cr_cntrl']

# --- 3. Exporting Results ---
# Saving the final analytical dataset for reporting and visualization
df_final.to_csv('ab_test_results_final.csv', index=False)

print("Success! File 'ab_test_results_final.csv' has been generated.")

Success! File 'ab_test_results_final.csv' has been generated.


Step 8: Data Sanitization & Final Integrity Check

Before concluding the analysis, we must handle potential mathematical anomalies such as division by zero, which results in infinite values (inf). This step ensures the dataset is clean and ready for visualization tools like Tableau, which might otherwise fail to process these records.

In [ ]:
# --- 1. Handling Mathematical Anomalies ---
# Replacing infinite values (from division by zero) with NaN,
# then filling all NaNs with 0 to ensure data stability.
df_final = df_final.replace([np.inf, -np.inf], np.nan).fillna(0)

# --- 2. Final Validation ---
# Verifying the integrity of the final dataset
print(f"Total rows processed: {len(df_final)}")
print(f"Are there any missing values? {df_final.isnull().values.any()}")

# Displaying the final result for a last look
df_final.head()

Total rows processed: 1952
Are there any missing values? False


,test,dimension_name,dimension_value,metric,session_cntrl,conversions_cntrl,session_test,conversions_test,p_value,significant,cr_cntrl,cr_test,uplift
0,1,country,(not set),add_payment_info,369,6,373,6,0.984979,False,0.01626,0.016086,-0.010724
1,1,country,Albania,add_payment_info,9,1,16,0,0.173568,False,0.111111,0.0,-1.0
2,1,country,Algeria,add_payment_info,29,1,23,1,0.866954,False,0.034483,0.043478,0.26087
3,1,country,Argentina,add_payment_info,122,2,122,2,1.000000,False,0.016393,0.016393,0.0
4,1,country,Armenia,add_payment_info,8,0,11,0,1.000000,False,0.0,0.0,NaN


Checking data types and memory

In [ ]:
# 1. Checking data types and memory
print("--- INFO ---")
print(df_final.info())

# 2. Checking for specific values ​​that Tableau doesn't like
import numpy as np
print("\n--- STATS ---")
stats = pd.DataFrame({
    'Nulls': df_final.isnull().sum(),
    'Infs': (df_final == np.inf).sum() + (df_final == -np.inf).sum(),
    'Zeros': (df_final == 0).sum(),
    'Unique_Vals': df_final.nunique()
})
print(stats)

# 3. Validating Boolean fields and business logic
print("\n--- BOOLEAN CHECK ---")
print(df_final['significant'].value_counts())

# 4. Checking problem rows (where there was division by zero)
print("\n--- PROBLEM ROWS (Uplift is NaN or 0) ---")
print(df_final[df_final['uplift'].isna() | (df_final['uplift'] == 0)].head(5))

--- INFO ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1952 entries, 0 to 1951
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   test               1952 non-null   Int64  
 1   dimension_name     1952 non-null   object 
 2   dimension_value    1952 non-null   object 
 3   metric             1952 non-null   object 
 4   session_cntrl      1952 non-null   Int64  
 5   conversions_cntrl  1952 non-null   Int64  
 6   session_test       1952 non-null   Int64  
 7   conversions_test   1952 non-null   Int64  
 8   p_value            1952 non-null   float64
 9   significant        1952 non-null   bool   
 10  cr_cntrl           1952 non-null   Float64
 11  cr_test            1952 non-null   Float64
 12  uplift             1952 non-null   Float64
dtypes: Float64(3), Int64(5), bool(1), float64(1), object(3)
memory usage: 200.3+ KB
None

--- STATS ---
                   Nulls  Infs  Zeros  Unique_Vals
test

Step 9: Sanitization

In [ ]:
import numpy as np

# Creating a copy for final processing
df_tableau = df_final.copy()

# 1. Correct the data types to standard ones (so that Tableau doesn't get confused)
# Converting from Pandas Int64/Float64 to regular numpy int/float
df_tableau['session_cntrl'] = df_tableau['session_cntrl'].astype(int)
df_tableau['conversions_cntrl'] = df_tableau['conversions_cntrl'].astype(int)
df_tableau['session_test'] = df_tableau['session_test'].astype(int)
df_tableau['conversions_test'] = df_tableau['conversions_test'].astype(int)

# 2. Uplift Processing (most important!)
# If the control was 0, and the test > 0, the uplift cannot be 0.
# In such cases, we usually set 1 (i.e. 100%) or leave a special label.
# For visualization in Tableau, it is better to replace 0 where there was a real increase:
mask_new_growth = (df_tableau['cr_cntrl'] == 0) & (df_tableau['cr_test'] > 0)
df_tableau.loc[mask_new_growth, 'uplift'] = df_tableau['cr_test'] # We set the test conversion value as a proxy for growth

# 3. Boolean у Integer (True -> 1, False -> 0)
df_tableau['significant'] = df_tableau['significant'].astype(int)

# 4. Format the p-value for convenience (rounding to 4 digits)
df_tableau['p_value'] = df_tableau['p_value'].round(4)

# 5. Remove any remaining NaNs if they have slipped in somewhere
df_tableau = df_tableau.fillna(0)

# Checking types before saving
print(df_tableau.dtypes)

# Export with utf-8-sig encoding (so that Excel and Tableau see Cyrillic/symbols correctly)
df_tableau.to_csv('ab_test_final_for_tableau.csv', index=False, encoding='utf-8-sig')

# Forcefully convert Float64 remainders to regular float
for col in ['cr_cntrl', 'cr_test', 'uplift']:
    df_tableau[col] = df_tableau[col].astype(float)

# Check again.
print(df_tableau[['cr_cntrl', 'cr_test', 'uplift']].dtypes)

# Overwrite the file
df_tableau.to_csv('ab_test_final_for_tableau.csv', index=False, encoding='utf-8-sig')

print("\nФайл 'ab_test_final_for_tableau.csv' готовий. Завантажуй його!")

test                   Int64
dimension_name        object
dimension_value       object
metric                object
session_cntrl          int64
conversions_cntrl      int64
session_test           int64
conversions_test       int64
p_value              float64
significant            int64
cr_cntrl             Float64
cr_test              Float64
uplift               Float64
dtype: object
cr_cntrl    float64
cr_test     float64
uplift      float64
dtype: object

Файл 'ab_test_final_for_tableau.csv' готовий. Завантажуй його!


In [ ]:
display(df_tableau)

,test,dimension_name,dimension_value,metric,session_cntrl,conversions_cntrl,session_test,conversions_test,p_value,significant,cr_cntrl,cr_test,uplift
0,1,country,(not set),add_payment_info,369,6,373,6,0.9850,0,0.016260,0.016086,-0.010724
1,1,country,Albania,add_payment_info,9,1,16,0,0.1736,0,0.111111,0.000000,-1.000000
2,1,country,Algeria,add_payment_info,29,1,23,1,0.8670,0,0.034483,0.043478,0.260870
3,1,country,Argentina,add_payment_info,122,2,122,2,1.0000,0,0.016393,0.016393,0.000000
4,1,country,Armenia,add_payment_info,8,0,11,0,1.0000,0,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,4,continent,Africa,new account,1096,96,1086,74,0.0901,0,0.087591,0.068140,-0.222069
1948,4,continent,Americas,new account,58053,4990,58031,4836,0.1087,0,0.085956,0.083335,-0.030494
1949,4,continent,Asia,new account,24908,2164,25165,2039,0.0182,1,0.086880,0.081025,-0.067386
1950,4,continent,Europe,new account,19693,1642,19528,1633,0.9305,0,0.083380,0.083624,0.002922
